# Homework 4
#### By Kunal Jha

## Question 2
We are given the following algorithm to find and return the majority element in the array if one exists:

In [7]:
# Pre-condition: n >= 1
# Post-condition: returns (false, nothing) if a[1...n] has no majority element
#                 returns (true, x) if x is the majority element of a[1...n]
function majority(a::Array{Integer,1} , n::Integer) # array of integers a of size n
    x = a[1]
    c = 1
    for i in 2:n
        if a[i] == x
            c = c + 1
        elseif c > 0
            c = c - 1
        else
            x = a[i]
            c = 1
        end
    end
    
    m = 0
    for j in 1:n # count number of occurences of c in a 
        if a[j] == x
            m = m + 1
        end
    end
    
    if m > n / 2
        return (true, x)
    else
        return (false, nothing)
    end
end

majority (generic function with 2 methods)

We are asked to verify the correctness of the algorithm

## Proof for Question 2
### Lemma 1 (The Loop-Invariant)
At the start of each iteration of the for-loop in lines 3-10, the following statements are true:

1. 2 $\le$ *i* $\le$ *i* + 1

2. 0 $\le$ *c* $\le$ *num*(*x*, *i* - 1) $\le$ *i* - 1

3. *x* $\in$ *a*[1 : *i*-1]

4. 2*num*(*x*, *i* - 1) - (*i* - 1) $\le$ *c*

5. $\forall$ *y* $\in$ *a*[1 : *i* - 1]: *y* $\neq$ *x* $\implies$ (*i* - 1) - 2*num*(*y*, *i* - 1) $\ge$ *c*

In more simple terms, the above invariant formalizes the idea that:
For an array *a* at the *i*th iteration,

1. c = 0 $\implies$ $\forall$ *y* $\in$ *a*[1: i - 1]: *y* is not a majority

2. c > 0 $\implies$ $\forall$ *y* $\in$ *a'*[1: i - 1]: *y* is not a majority, 
    where *a'* = {array *a* - *c* elements of *x* in *a*}

### Lemma 2 (Algorithm's correctness)
The for-loop terminates at the start of the iteration when *i* = *n* + 1. Substituting *i* = *n* + 1, we have two cases:

1. If c = 0, then we know that all of the elements in *a*[1 : n] are not a majority, since 
    Part (4) of the loop invariant says *num*(*x*, *n*) $\le$ $\frac{c + n}{2}$  = $\frac{n}{2}$, and 
    Part (5) says $\forall$ *y* $\in$ *a*[1 : *n*]: *y* $\neq$ *x* $\implies$ $\frac{n - c}{2}$ = $\frac{n}{2}$ $\ge$ *num*(*y*, *n*). Thus, the function will return (false, nothing), as no majority exists.

2. If c > 0, then we know all of the elements in *a*[1 : n] that are not equal to x aren't a majority, since
    Part (5) says $\forall$ *y* $\in$ *a*[1 : *n*]: *y* $\neq$ *x* $\implies$ $\frac{n}{2}$ > $\frac{n - c}{2}$ $\ge$ num*(*y*, *n*). Thus, when we count the occurences of *x* in *a*[1 : n] and store it as the variable *m*, checking to see if *m* > *n*/2 will tell us whether or not a majority exists. If *m* > *n*/2, then *x* is a majority since it occurs at least *n*/2 times in the array, so (true, *x*) is returned. Otherwise, (false, nothing) is returned since *x* is not a majority element and we already proved the other elements distinct from x in the array *a*[1 : *n*] are not majorities.

Hence, the post-condition is satisfied, and the algorithm's correctness is verified. $\blacksquare$

## Question 4
We are asked to write a recursive program which finds and returns the local minimum in an $n \times n$ array if one exists, and this program must run in *O(n)* time.

### Part 1: Pre/Post Conditions and Algorithm

### Helper Methods

In [67]:
# Pre-condition: G must be an n x m matrix for some natural numbers n, m
#                rIdx is a natural number such that 1 <= rIdx <= n
#                p and q are natural numbers such that 1 <= p <= q <= m
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= m, 
#                 G[x, y] is the smallest element in the rIdx'th row of G from range p:q columns
function minRow(G::Array{Float64, 2}, p::Integer, q::Integer, rIdx::Integer)
    min = G[rIdx, p]
    minY = p
    for i in p+1:q
        if G[rIdx, i] < min
            minY = i
        end
    return (rIdx, minY)
    end
end

# Pre-condition: G must be an n x m matrix for some natural numbers n, m
#                cIdx is a natural number such that 1 <= cIdx <= m
#                p and q are natural numbers such that 1 <= p <= q <= n
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= m, 
#                 G[x, y] is the smallest element in the cIdx'th column of G from range p:q rows
function minCol(G::Array{Float64, 2}, p, q, cIdx::Integer)
    min = G[p, cIdx]
    minX = p
    for i in p+1:q
        if G[i, cIdx] < min
            minX = i
        end
    return (minX, cIdx)
    end
end

# Pre-condition: G must be an n x m matrix for some natural numbers n, m
#                i,j,p,q are natural numbers such that 1 <= i <= j <= n, and 1 <= p <= q <= m
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= m, 
#                 G[x, y] is the smallest element in the border surrounding the (j-i+1) x (q-p+1) sub-matrix
function minBorder(G::Array{Float64, 2}, i, j, p, q)
    minEle = G[i, p]
    min = (i, p)
    for idx in i+1:j # check top row
        if G[idx, p] < minEle
            minEle = G[idx, p]
            min = (idx, p)
        end
    end
    for idx2 in i:j # check bottom row
        if G[idx2, q] < minEle
            minEle = G[idx2, q]
            min = (idx2, q)
        end
    end
    for idx3 in p:q # check left column
        if G[i, idx3] < minEle
            minEle = G[i, idx3]
            min = (i, idx3)
        end
    end
    for idx4 in p:q # check right column
        if G[j, idx4] < minEle
            minEle = G[j, idx4]
            min = (j, idx4)
        end
    end
    return min # return min
end

# Pre-condition: G must be an n x n matrix for some natural numbers n
#                i,j,p,q are natural numbers such that 1 <= i <= j <= n, and 1 <= p <= q <= n
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= n
#                 G[x, y] is the minimum value of 
#                 the minimum row element for the n/2'th row, the minimum column element for the n/2'th column,
#                 and the minimum border element over the subarray
function minOverall(G::Array{Float64, 2}, i, j, p, q)
    med = (j-i+1)/2 + i
    smallCol = minCol(G, i, j, med+i)
    smallRow = minRow(G, p, q, med+p)
    smallBorder = minBorder(G, i, j, p, q)
    # compare the three quantities and return the smallest
    if G[smallCol[1],smallCol[2]] < G[smallRow[1],smallRow[2]] && G[smallCol[1],smallCol[2]] < G[smallBorder[1],smallBorder[2]]
        return smallCol
    elseif G[smallRow[1],smallRow[2]] < G[smallCol[1],smallCol[2]] && G[smallRow[1],smallRow[2]] < G[smallBorder[1],smallBorder[2]]
        return smallRow
    else
        return smallBorder
    end
end     

# Pre-condition: G must be an n x m matrix for some natural numbers n, m, 
#                curr must be a tuple such that 1 <= curr[1] <= n, 1 <= curr[2] <= m
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= m
#                 G[x, y] is the smallest neighbor of G[curr[1], curr[2]]. 
#                 If G[curr[1], curr[2]] is smaller than all of its neighbors, curr is returned
function findNext(G::Array{Float64, 2}, curr::Tuple{Int, Int})
    x = curr[1]
    y = curr[2]
    
    n = size(G,1) # number of rows in G
    m = size(G,2) # number of columns in G
    
    lMin = G[x,y]
    next = curr
    if (x > 1) && (lMin > G[x-1,y]) # compare to left neighbor
        lMin = G[x-1,y]
        next = (x-1,y)
    end
    if (y > 1) && (lMin > G[x,y-1]) # compare to bottom neighbor
        lMin = G[x,y-1]
        next = (x,y-1)
    end
    if (x < n) && (lMin > G[x+1,y]) # compare to right neighbor
        lMin = G[x+1,y]
        next = (x+1,y)
    end
    if (y < m) && (lMin > G[x,y+1]) # compare to top neighbor
        lMin = G[x,y+1]
        next = (x,y+1)
    end
    return next # return next smallest
end

findNext (generic function with 2 methods)

### Main Method

In [66]:
# Pre-condition: G is a n x n matrix with distinct numbers
#                i,j,p,q are natural numbers such that 1 <= i <= j <= n, and 1 <= p <= q <= m
# Post-condition: returns G[x, y], which is smaller than all of it's neighbors (it is a local min)
#.                for some (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= n

function find_local_min(G::Array{Float64, 2}, i, j, p, q)
    minCand = minOverall(G, i, j, p, q) # find the minimum across the cross and border
    next = findNext(G, minCand) # find the next smallest value given this
    med = ((j - i + 1) / 2)
    if minCand == next # if the current is already a local min, return that
        return G[minCand[1], minCand[2]]
    elseif (j - next[1]) > (next[1] - i)  # if the next is in the left half
        if (q - next[2]) > (next[2] - p) # top quadrant
            return find_local_min(G, i, i + med, p, p + med)
        else # bottom quadrant
            return find_local_min(G, i, i + med, p+med, q)
        end
    else # if the next is in the right half
        if (q - next[2]) > (next[2] - p) # top quadrant
            return find_local_min(G, i+med, j, p, p + med)
        else # bottom quadrant
            return find_local_min(G, i+med, j, p+med, q)
        end
    end
end

# Pre-condition: G is a n x n matrix with distinct numbers
# Post-condition: (x, y) where x and y are natural numbers such that 1 <= x <= n, 1 <= y <= n
#                 G[x, y] is smaller than all of it's neighbors (it is a local min)
function get_local_min(G::Array{Float64, 2})
    n = size(G,1) # number of rows/cols
    return find_local_min(G, 1, n, 1, n) # start with whole array
end

get_local_min (generic function with 1 method)

### Part 2: Proof of Method Correctness

#### Proving minRow function
The loop invariant is that *p* + 1 $\le$ i $\le$ *q* + 1 and that *minY* is the column index of smallest element in the row
*G*[*rIdx*, *p* : *i*-1]. It is also that *min* is the value of the smallest element in the row *G*[*rIdx*, *p* : *i*-1]. Substituting *i* = *q* + 1 into the second part of the loop invariant we see that *minY* is the smallest element in the row *G*[*rIdx*, *p* : *q*], which satisfies the post-condition when the method terminates and (*rIdx*, *minY*) is returned. Hence the claim. 
$\blacksquare$

#### Proving minCol function
The loop invariant is that *p* + 1 $\le$ i $\le$ *q* + 1 and that *minX* is the row index of the smallest element in the column 
*G*[*p* : *i*-1, *cIdx*]. It is also that *min* is the value of the smallest element in the column *G*[*p* : *i*-1, *cIdx*]. Substituting *i* = *q* + 1 into the second part of the loop invariant we see that *minY* is the index of the smallest element in the row *G*[*p* : *q*, *cIdx*], which satisfies the post-condition when the method terminates and (*minX*, *cIdx*) is returned. Hence the claim. 
$\blacksquare$

#### Proving minBorder function
The first loop's invariant is that *i* + 1 $\le$ idx $\le$ *j* + 1 and that *min* is the *row* and *col* indices of smallest element in the row
*G*[*i*:*idx*-1, *p*]. It is also that *minEle* is the value of the smallest element in the row *G*[*rIdx*, *p* : *i*-1]. Substituting *i* = *j* + 1 into the second part of the loop invariant we see that *min* is the indices of the smallest element in the row *G*[*i*:*j*, *p*].

The second loop's invariant is that *i* $\le$ idx2 $\le$ *j* + 1 and that *min* is the *row* and *col* indices of smallest element in the row
*G*[*i*:*j*, *p*] and the row *G*[*i*:*idx2*-1, *q*]. It is also that *minEle* is the value of the smallest element in the rows *G*[*i*:*j*, *p*] and the row *G*[*i*:*idx2*-1, *q*]. Substituting *idx2* = *j* + 1 into the second part of the loop invariant we see that *min* is the indices of the smallest element in the row *G*[*i*:*j*, *p*] and the row *G*[*i*:*j*, *q*].

The third loop's invariant is that *p* $\le$ idx3 $\le$ *q* + 1 and that *min* is the *row* and *col* indices of smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], and the column *G*[*i*, *p*:*idx3*-1]. It is also that *minEle* is the value of the smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], and the column *G*[*i*, *p*:*idx3*-1]. Substituting *idx3* = *q* + 1 into the second part of the loop invariant we see that *min* is the indices of the smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], and the column *G*[*i*, *p*:*q*]. 

The fourth loop's invariant is that *p* $\le$ idx4 $\le$ *q* + 1 and that *min* is the *row* and *col* indices of smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], the column *G*[*i*, *p*:*q*], and the column *G*[*j*, *p*:*idx4*-1]. It is also that *minEle* is the value of the smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], the column *G*[*i*, *p*:*q*], and the column *G*[*j*, *p*:*idx4*-1]. Substituting *idx4* = *q* + 1 into the second part of the loop invariant we see that *min* is the indices of the smallest element in the row
*G*[*i*:*j*, *p*], the row *G*[*i*:*j*, *q*], the column *G*[*i*, *p*:*q*], and the column *G*[*j*, *p*:*q*].

By the end of the fourth loop we have determiend the indices of the smallest item along the border of a matrix, which satisfies the post-condition when the loop terminates and *min* is returned. Hence the claim.
$\blacksquare$

#### Proving minOverall function
This function begins by calculating the midpoint of the array (number of elements / 2). It then uses this calculation to determine the smallest element in the middle column, the smallest element in the middle row, and the smallest element on the border. It does so by making calls to minCol, minRow, and minBorder respectively, which we know to work and return the correct answers due to the proofs above. Once It has obtained the indices for the smallest elements in each of the 3 aforementioned categories, it compares them with a simple series of if statements assessing the values of the indices in the matrix G. Once this comparison is done, the pair of indices which result in the lowest value in G are returned. This satisfies the post-condition when the comparisons are conducted and the indices with the minimum value are returned. Hence the claim.
$\blacksquare$

#### Proving findNext function
This function finds the next smallest neighbor of an element in the matrix G if one exists. It does so by checking each of the 4 neighbors in the matrix (if they exist within the matrix's dimensions) and seeing if it is less than the minimum. If a neighbor is less than the current minimum, that neighbor's indices are stored and that neighbor becomes the new minimum. We are essentially keeping a running minimum and iterating through the list of neighbors, updating the minimum if any neighbor is less than it. If none of the conditionals are executed, this means the current value is the local minimum, so it's indices are not updated and instead returned as is. Thus, the post-condition is satisfied when at most 4 conditional statements are passed through and terminates, returning the indices of the next smallest element in the matrix G. Hence the claim.
$\blacksquare$

#### Proving find_local_min function
Let *P(n)* be a predicate which says, "If find_local_min(G,i,j,p,q) is called with *n* = *j* − *i* + 1 = *p* - *q* + 1  and the pre-condition of the method satisfied, then the method terminates and, when it terminates, the post-condition is satisfied.

Claim: $\forall$*n* $\ge$ 1: *P(n)*

Proof: By Induction
* Base Case: Suppose the method is called with n = 1 and the pre-condition is satisfied. Then we know that in a $1 \times 1$ matrix, the only element within the 2d array can be trivially called the local minimum. Thus, lines 1 and 2 result in the same set of indices, which line 5 returns. Hence, the base case holds.

* Inductive Step: Let *k* $\ge$ 1 be arbitrary and assume that *P*(1), *P*(2), ..., *P*(k −1) are true. Since line 1 determined *minCand*, the current smallest element between the middle row, middle column, and the border, and line 2 determined *next*, the smallest neighbor of the element found in line 1, we see 2 cases:
    1. *next* = *minCand* : in this case, *minCand* is returned in line 5 since it is not adjacent to a smaller value, implying it is the local minimum.
    2. *next* < *minCand* : in this case, we know that *next* lies in a $k/2 \times k/2$ quadrant of G, such that if G was split into 4 even, distinct sections of these dimensions, *next* would be found in one of them. Because we know that *next* is smaller than an adjacent element found either at the cross-section of the middle of the matrix or along the border, we know that the quadrant it resides in must have a local minimum. You may be wondering where does this intuition come from? Well it derives from the realization that if there exists an element within a 2d array of distinct elements such that it is smaller than the minimum value of that array's border, that array will have a local minimum (because with any array of distinct elements, the existence of a pure minimum in that array implies the existance of a local minimum). Thus, the elseif and else statements isolate the specific $k/2 \times k/2$ quadrant this adjacent point lies in and recurses on that, including the section of the original border and cross-section in the sub-matrix. We further boost our confidence of recursing on the correct sub-array by the observation that if the local minimum was in a separate sub-array, you would have to cross the middle column or middle row to access it. However, we have already picked the sub-matrix with the smallest bordering value and checked to make sure it wasn't a local minimum, so switching the border would be impossible.
    Since we pass in a square matrix with valid indices to illustrate the submatrix being examined without creating copies of the original array, we have satisfied the pre-condition. By the inductive hypothesis, *P*($\frac{k}{2}$) is true, so we know that when the program terminates, it will return the value of the local minimum in the matrix G of $k \times k$ dimensions. Thus, we have satisfied the post-condition and shown *P*(k). 

Hence the inductive step is complete. 

The claim then follows from the principles of strong induction.
$\blacksquare$

The *get_local_min* function is proved correctly because it simply makes a call to the *find_local_min* function and instantiates it's search area to include the entire matrix. I have not included this proof because it is trivially true under the same evidence as in the previous proof.

### Part 3: Time Complexity Analysis

The recurrence for the *find_local_min* function is *T*(n) $\le$ *T*($\frac{n}{2}$) + *O*(*n*), where *n* is the number of rows in an $n \times n$ matrix. This is because as n approaches a sufficiently large quantity, the *minOverall* operation takes *O*(*n*) time, the *findNext* operation takes *O*(1) time, and you are recursing on a matrix of size $$\frac{n}{2} \times \frac{n}{2}$$. 

We know that *minOverall* takes *O*(*n*) time because it calls the *minCol*, *minRow*, and *minBorder* functions. Since all of these operations do $\Theta$(1) operations on a single for loops rather than nested ones, and each of these for loops are contingent on the size of n, we can say that all of them operate in *O*(*n*) time, so taking their results sequentially can also be done in *O*(*n*) time. 

*findNext* takes *O*(1) time since you are doing at most 4, $\Theta$(1) operations. 

Rewriting the recurrence, we get *T*(n) $\le$ *T*($\frac{n}{2}$) + *an*, for some *a* > 0
Now that we know the recurrence for the *find_local_min* function, we use the Master Method to see that

$n^k$ = $n^.01$ $\le$ *an*, where *k* = log<sub>2</sub>1 + 0.01, and *a* = 1

Moreover, we see that

$\frac{n}{2}$ $\le$ *cn*, if *c* = 1

Since we have shown $n^k$ = $\le$ *dn*, where *k* = log<sub>b</sub>a + e, such that *a* = 1, *b* = 2, for some *d* > 0, *e* > 0, as well as the regularity condition *af*($\frac{n}{b}$) $\le$ *df*(*n*), for some *d* > 0, case 3 of the Master Method applies and 

*T*(n) = *O*(*n*)
$\blacksquare$